In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import warnings 
warnings.filterwarnings("ignore")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/churn-modelling/Churn_Modelling.csv", index_col = "RowNumber")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.Geography.unique()

In [ ]:
df.Gender.unique()


In [ ]:
df.Tenure.unique()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.drop(["CustomerId","Surname"], axis = 1, inplace = True)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#Numerical Variables
num_var = [f for f in df.columns if df[f].dtype!='O']
num_var


In [ ]:
#Discrete variables 
dis_var = [f for f in num_var if df[f].nunique() < 20]
dis_var

In [ ]:
df.Tenure.nunique()

In [ ]:
#Continous variable
cont_var = [f for f in num_var if f not in dis_var]
cont_var

In [ ]:
#Categorical Variables
cat_var = [f for f in df.columns if f not in num_var]
cat_var


# EDA

In [ ]:
for var in dis_var:
    sns.countplot(df[var])
    plt.show()

Inference : The Dataset is imbalanced (Exited column)


In [ ]:
for var in cont_var:
    sns.distplot(df[var], kde = False, color = "green")
    plt.show()
    sns.boxplot(df[var])
    plt.show()

Inference : 

The data does not follow any distribution (Need to convert to gaussian)

There are outliers in CreditScore and Age columns

In [ ]:
for var in cat_var:
    sns.countplot(df[var])
    plt.show()

In [ ]:
plt.figure(figsize = (20,10))
sns.heatmap(df.corr(), annot =True)

The target variable(exited) is 
positively correlated with - Age, Balance,Estimated Salary   (Weak)

Negatively with - CreditScore,Tenure,NumofProducts,CreditCard,IsActiveMember

In [ ]:
plt.figure(figsize = (20,10))
sns.pairplot(df)

# Handling Imbalanced Dataset 

In [ ]:
exited_yes = df[df["Exited"]==1]
exited_no = df[df["Exited"]==0]

In [ ]:
print(exited_yes.shape,exited_no.shape)

In [ ]:
x = df.drop(["Exited"], axis = 1)
y = df["Exited"]

In [ ]:
x.head()
y.head()

In [ ]:
cat_1 = pd.get_dummies(df["Geography"], drop_first = True)
cat_2 = pd.get_dummies(df["Gender"], drop_first = True)

In [ ]:
x.head()

In [ ]:
x_t = pd.concat([x,cat_1],axis=1)
x_t_f = pd.concat([x_t,cat_2],axis = 1)
x_t_f.head()

In [ ]:
x_t_f = x_t_f.drop(["Geography","Gender"],axis =1)

In [ ]:
x_t_f.head()

In [ ]:
y.head()

In [ ]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE(k_neighbors=4)
x_res, y_res = oversample.fit_resample(x_t_f, y)

In [ ]:
print(x_res.shape,y_res.shape)

# Scaling the features

In [ ]:
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()

In [ ]:
x_res = scalar.fit_transform(x_res)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x_res,y_res,test_size = 0.25)

# Model Selection and HyperParamter Tuning

In [ ]:
params={
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
    
}

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
classifier = XGBClassifier()
random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)
random_search.fit(x_train,y_train)

In [ ]:
random_search.best_estimator_

In [ ]:
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
clf1 = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.1, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.2, max_delta_step=0, max_depth=8,
              min_child_weight=3, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
clf1.fit(x_train,y_train)
print(clf1.score(x_test,y_test))
